In [57]:
# Import Necessary Libraries for data acquistion and prep
import pandas as pd
from env import get_db_url
import os

# Set options so that we can view all the data
pd.options.display.max_columns = None
pd.options.display.max_rows = None

1.  **Acquisition-Gather**: 

Gather structured data from SQL to Pandas

In [28]:
def get_zillow_data(use_cache=True):
    '''
    This functions recieves a boolean as input to see if the user wants to recieve a fresh copy from the database.
    Then the fucntion checks to see if the file being requested already exists.
    Runs a query for the data using the assigned url.
    Creates a new csv if needed.
    Then returns the zillow dataframe.
    '''
    
    # Assign filename to csv for storage
    filename = 'zillow.csv'
    
    # Check if file exists and if user wants a fresh copy from the database
    if os.path.exists(filename) and use_cache:
        print('Using cached csv file...')
        return pd.read_csv(filename)
    
    # Notify user of next step
    print('Retrieving from database...')
    
    # Assign url
    url = get_db_url('zillow')
    
    # Run query for data
    zillow_data = pd.read_sql('''
    SELECT *
    FROM properties_2017
    JOIN predictions_2017
    USING(parcelid)
    JOIN propertylandusetype
    USING(propertylandusetypeid)
    WHERE propertylandusetypeid = 261 & 
    
    ''', url)
    
    # Notify user of next step
    print('Saving new csv...')
    
    # Create csv
    zillow_data.to_csv(filename, index=False)
    
    # Return DataFrame
    return zillow_data

In [29]:
# Call the above function and assign the dataframe a variable name
zillow = get_zillow_data(False)

Getting a fresh copy from the database...
Saving to csv...


In [ ]:
# Assign a url to call the other identifying tables
url = get_db_url('zillow')

# Get identifying tables
air = pd.read_sql('''SELECT * FROM airconditioningtype''', url)
arch = pd.read_sql('''SELECT * FROM architecturalstyletype''', url)
building_class = pd.read_sql('''SELECT * FROM buildingclasstype''', url)
heat = pd.read_sql('''SELECT * FROM heatingorsystemtype''', url)
use = pd.read_sql('''SELECT * FROM propertylandusetype''', url)
story = pd.read_sql('''SELECT * FROM storytype''', url)
construction = pd.read_sql('''SELECT * FROM typeconstructiontype''', url)

# Just out of curiousity
unique_prop = pd.read_sql('''SELECT * FROM unique_properties''', url)

# Name supplemental dataframes
dfs = ['air','arch','building_class','heat','use','story','construction']

2.  **Acquisition-Summarize**: 

Summarize the data through aggregates, descriptive stats and distribution plots (histograms, density plots, boxplots, e.g.). (pandas: `.value_counts`, `.head`, `.shape`, `.describe`, `.info`, `matplotlib.pyplot.hist`, `seaborn.boxplot`)

In [58]:
zillow.head()

,propertylandusetypeid,parcelid,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id,logerror,transactiondate,propertylandusedesc
0,261.0,14297519,1727539,NaN,NaN,NaN,3.5,4.0,None,NaN,3.5,NaN,NaN,3100.0,3100.0,None,None,NaN,NaN,6059.0,NaN,3.0,2.0,633.0,NaN,NaN,33634931.0,-117869207.0,4506.0,NaN,NaN,NaN,NaN,NaN,122,None,6.059063e+07,53571.0,1286.0,NaN,96978.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN,1998.0,NaN,NaN,485713.0,1023282.0,2016.0,537569.0,11013.72,None,NaN,6.059063e+13,0,0.025595,2017-01-01,Single Family Residential
1,261.0,17052889,1387261,NaN,NaN,NaN,1.0,2.0,None,NaN,1.0,NaN,1465.0,1465.0,1465.0,None,None,1465.0,NaN,6111.0,1.0,1.0,1.0,0.0,NaN,NaN,34449266.0,-119281531.0,12647.0,NaN,NaN,NaN,NaN,NaN,1110,None,6.111001e+07,13091.0,2061.0,NaN,97099.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,1967.0,1.0,NaN,88000.0,464000.0,2016.0,376000.0,5672.48,None,NaN,6.111001e+13,1,0.055619,2017-01-01,Single Family Residential
2,261.0,14186244,11677,NaN,NaN,NaN,2.0,3.0,None,NaN,2.0,NaN,NaN,1243.0,1243.0,None,None,NaN,NaN,6059.0,NaN,2.0,2.0,440.0,NaN,NaN,33886168.0,-117823170.0,8432.0,1.0,NaN,NaN,NaN,1.0,122,None,6.059022e+07,21412.0,1286.0,NaN,97078.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,1962.0,1.0,NaN,85289.0,564778.0,2016.0,479489.0,6488.30,None,NaN,6.059022e+13,2,0.005383,2017-01-01,Single Family Residential
3,261.0,12177905,2288172,NaN,NaN,NaN,3.0,4.0,None,8.0,3.0,NaN,NaN,2376.0,2376.0,None,None,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,2.0,34245180.0,-118240722.0,13038.0,1.0,NaN,NaN,NaN,1.0,0101,LCR110000*,6.037300e+07,396551.0,3101.0,NaN,96330.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1970.0,NaN,NaN,108918.0,145143.0,2016.0,36225.0,1777.51,None,NaN,6.037300e+13,3,-0.103410,2017-01-01,Single Family Residential
4,261.0,12095076,781532,1.0,NaN,NaN,3.0,4.0,None,9.0,3.0,NaN,NaN,2962.0,2962.0,None,None,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,2.0,34145202.0,-118179824.0,63000.0,1.0,NaN,NaN,NaN,1.0,0101,PSR2,6.037461e+07,47019.0,3101.0,274684.0,96293.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1950.0,NaN,NaN,276684.0,773303.0,2016.0,496619.0,9516.26,None,NaN,6.037461e+13,6,-0.001011,2017-01-01,Single Family Residential


In [78]:
# Check Datatypes and nulls
# zillow.info(null_counts=True)

In [76]:
# zillow.isnull().sum()

- Lots of Null values. Some Object values, mostly floats

In [86]:
# make df for analysis with dropped parcelid and id to check values 
zillow_ex = zillow.copy()
zillow_ex = zillow_ex.drop(columns=['parcelid','id'])

# loop through columns that are not floats (continuous) and look at value counts
#zillow_ex.columns[zillow_ex.dtypes != 'float']
# for col in zillow_ex.columns[zillow_ex.dtypes != 'float']:
#     print(f'{col}\n\n{zillow_ex[col].value_counts(dropna=False, ascending=True)}\n\n')
# zillow_ex.columns

- buildingclassid, finishedsquarefeet13, finsishedsquarefeet15 all null
- propertylandusedesc are all the same value 
- taxdeliquencyflag has 50363 nulls
- transactiondate has 1 2018 value
- Drop all above

In [80]:
zillow_ex = zillow_ex.drop(columns=['buildingclasstypeid','finishedsquarefeet13','finishedsquarefeet15','propertylandusedesc','taxdelinquencyflag',])

3.  **Preparation-Clean**: 

We will convert datatypes and handle missing values. In this module we will keep it simple in how we handle missing values. We will introduce other ways to handle missing values as we progress through the course. (pandas: `.isnull`, `.value_counts`, `.dropna`, `.replace`)

4.  **Preparation-Split**:

We will sample the data so that we are only using part of our available data to analyze and model. We will discuss the reasons for doing this. This is known as "Train, Validate, Test Splitting". (`sklearn.model_selection.train_test_split`).

5.  **Preparation-Scale**: 

We will discuss the importance of "scaling" data, i.e. putting variables of different units onto the same scale. We will scale data of different units to be on the same scale so they can be compared and modeled. We will discuss different methods for scaling data and why to use one type over another. (`sklearn.preprocessing`: `StandardScaler`, `QuantileTransformer`, `PowerTransformer`, `RobustScaler`, `MinMaxScaler`)

6.  **Exploration-Hypothesize**:

We will discuss the meaning of "drivers", variables vs. features, and the target variable. We will disucss the importance of documenting questions and hypotheses, obtaining answers for those questions, and documenting takeaways and findings at each step of exploration.

7.  **Exploration-Visualize**: 

We will use visualization techniques (scatterplot, jointplot, pairgrid, heatmap) to identify drivers. When a visualization needs to be followed up with a test, we will do so.

8.  **Exploration-Test**: 

We will analyze the drivers of a continuous variable using appropriate statistical tests (t-tests and correlation tests).

9.  **Modeling-Feature Engineering**:

We will learn ways to identify, select, and create features through feature engineering methods, specifically feature importance. We will discuss the "Curse of Dimensionality." (`sklearn.feature_selection.f_regression`).

10.  **Modeling-Establish Baseline**: 

We will learn about the importance of establishing a "baseline model" or baseline score and ways to complete this task.

11.  **Modeling-Build Models**: 

We will build linear regression models, i.e. we will use well established algorithms, such as glm (generalized linear model) or a basic linear regression algorithm (e.g. y = mx + b), to extract the patterns the data is demonstrating and return to us a mathematical model or function (e.g. y = 3x + 2) that will predict the target variable or outcome we want to predict. We will learn about the differences in the most common regression algorithms. (`sklearn.linear_model`)

12.  **Modeling-Model Evaluation**:

We will compare regression models by computing "evaluation metrics", i.e. metrics that measure how well a model did at predicting the target variable. (`statsmodels.formula.api.ols`, `sklearn.metrics`, `math.sqrt`)

13.  **Modeling-Model Selection and Testing**: 

We will learn how to select a model, and we will test the model on the unseen data sample (the out-of-sample data in the validate and then test datasets).

14.  **Data Science Pipeline and Product Delivery**: 

We will end with an end-to-end project practicing steps of the data science pipeline from planning through model selection and delivery.